In [1]:
!pip install langchain_community transformers pypdf

In [2]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [3]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import BartForConditionalGeneration, BartTokenizer, pipeline

# Load saved model from Drive
model_path = "/content/drive/MyDrive/cnn-summ"
model = BartForConditionalGeneration.from_pretrained(model_path)
tokenizer = BartTokenizer.from_pretrained(model_path)
summarizer = pipeline("summarization", model=model, tokenizer=tokenizer)


Device set to use cuda:0


In [4]:
# Load and split PDF
def extract_pdf_chunks(file_path):
    loader = PyPDFLoader(file_path)
    pages = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    chunks = text_splitter.split_documents(pages)

    return [chunk.page_content for chunk in chunks]


In [14]:
# Summarize each chunk and combine
def summarize_large_pdf(file_path):
    chunks = extract_pdf_chunks(file_path)
    all_summaries = []

    for i, chunk in enumerate(chunks):
        try:
            input_length = len(tokenizer.encode(chunk))
            max_len = max(30, input_length // 3)

            summary = summarizer(chunk, max_length=max_len, min_length=15, do_sample=False)

            all_summaries.append(summary[0]['summary_text'])
        except Exception as e:
            print(f"Error summarizing chunk {i}: {e}")

    return " ".join(all_summaries)


In [15]:
# Example usage
pdf_path = "/content/drive/MyDrive/environment.pdf"
final_summary = summarize_large_pdf(pdf_path)
print("Final Summary:\n", final_summary)

Your max_length is set to 30, but your input_length is only 8. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=4)


Final Summary:
 Environment means different things to different people. To some it means home: to others it may refer to a village, a city a country or the  whole world. It is formed by combining two words, viz. ‘environ’ and ‘ment’ meaning ‘encircle’ or ‘all round’ Environmental studies discipline has multiple and multilevel scopes. This study is important and necessary not only for children but also foe everyone: literate or illiterate. Environmental study is based upon a comprehensive view of various environmental systems. To know the necessity of conservation of natural resources is important. World population is increasing at an alarming rate especially in developing countries. Resources are over-exploited and there is no foresight of leaving the resources to the future. Education and training are needed to save the biodiversity and species extinction. The United Nations Conference on Environment and Development held at Reo de Janeiro in 1992 (popularly known as ‘Earth Summit ‟) a